## Contetnts
- 기초 연산
- 그룹 연산 : 집계, 변환, 필터링, 그룹매핑
- 함수 매핑 : map, apply, applymap
- Pivot

In [ ]:
# 기초 연산

import pandas as pd

st1 = pd.Series({'국어':100,'영어':80,'수학':90})
st2 = pd.Series({'수학':80,'국어':90,'영어':80})

print(st1,'\n')
print(st2)

국어    100
영어     80
수학     90
dtype: int64 

수학    80
국어    90
영어    80
dtype: int64


In [ ]:
pd.DataFrame(st1+st2)


,0
국어,190
수학,170
영어,160


In [ ]:
add = st1 + st2
sub = st1 - st2
mul = st1 * st2
div = round((st1/st2),2)

# concat : 국어, 수학, 영어가 행으로 들어간다 (index)
df = pd.concat([add, sub, mul, div], axis = 1)
df.columns=['덧셈', '뺄셈', '곱셈', '나눗셈']
df

,덧셈,뺄셈,곱셈,나눗셈
국어,190,10,9000,1.11
수학,170,10,7200,1.12
영어,160,0,6400,1.00


In [ ]:
# 국어, 수학, 영어가 열로 들어간다 (columns)
res = pd.DataFrame([add,sub,mul,div])
res.index = ['덧셈', '뺄셈', '곱셈', '나눗셈']
res

,국어,수학,영어
덧셈,190.00,170.00,160.0
뺄셈,10.00,10.00,0.0
곱셈,9000.00,7200.00,6400.0
나눗셈,1.11,1.12,1.0


In [ ]:
import numpy as np

st1 = pd.Series({'국어':np.nan,'영어':80,'수학':90})
st2 = pd.Series({'수학':80,'국어':90})

add = st1 + st2
sub = st1 - st2
mul = st1 * st2
div = round((st1 / st2),2)

result = pd.DataFrame([add,sub,mul,div],
                     index=['덧셈','뺄셈','곱셈','나눗셈'])
result

,국어,수학,영어
덧셈,NaN,170.00,NaN
뺄셈,NaN,10.00,NaN
곱셈,NaN,7200.00,NaN
나눗셈,NaN,1.12,NaN


In [ ]:
# Q. Null 값을 0으로 치환
res = result.copy()
result.fillna(0, inplace=True)
result

res['국어'].fillna(0)
res['영어'].fillna(0)
res

st1 = pd.Series({'국어':np.nan,'영어':80,'수학':90})
st2 = pd.Series({'수학':80,'국어':90})
ad = st1.add(st2, fill_value=0)
su = st1.sub(st2, fill_value=0)
mu = st1.mul(st2, fill_value=0)
di = st1.div(st2, fill_value=0)

df = pd.DataFrame([ad,su,mu,di], index=['덧셈','뺄셈','곱셈','나눗셈'])
df

,국어,수학,영어
덧셈,90.0,170.000,80.0
뺄셈,-90.0,10.000,80.0
곱셈,0.0,7200.000,0.0
나눗셈,0.0,1.125,inf


In [ ]:
# 0~11까지 float 타입으로
np.arange(12.)
# 3행 4열 2차원 배열로
np.arange(12.).reshape((3,4))

array([[ 0.,  1.,  2.,  3.],
       [ 4.,  5.,  6.,  7.],
       [ 8.,  9., 10., 11.]])

In [ ]:
df1 = pd.DataFrame(np.arange(12.).reshape((3,4)), columns=list('abcd'))
df2 = pd.DataFrame(np.arange(20.).reshape((4,5)), columns=list('abcde'))

df2.loc[1, 'b'] = np.nan

df1+df2

,a,b,c,d,e
0,0.0,2.0,4.0,6.0,NaN
1,9.0,NaN,13.0,15.0,NaN
2,18.0,20.0,22.0,24.0,NaN
3,NaN,NaN,NaN,NaN,NaN


In [ ]:
# 계산 인자를 뒤집어 계산 : 원래라면 df1 - 인자값 이겠으나 r을 붙이면 반대로 인자값 - df1이된다.
df1.rdiv(1) # 1 / df1

df1.rsub(1) # 1 - df1

df1.radd(1) # 1 + df1

df1.rmul(1)

,a,b,c,d
0,0.0,1.0,2.0,3.0
1,4.0,5.0,6.0,7.0
2,8.0,9.0,10.0,11.0


In [ ]:
# 배열의 연산
arr = np.arange(12.).reshape((3,4))

# broadcasting : 전체 배열에 하나의 행 연산
# 일반 리스트에서는 적용안됨
arr - arr[0]

# DataFrame과 Series간 연산
frame = pd.DataFrame(np.arange(12.).reshape(4,3),columns=list('bde'),
                    index = ['Utah','Ohio','Texas','Oregon'])
frame - frame.iloc[0]

,b,d,e
Utah,0.0,0.0,0.0
Ohio,3.0,3.0,3.0
Texas,6.0,6.0,6.0
Oregon,9.0,9.0,9.0


In [ ]:
# 1줄인 2차원 배열
a = np.arange(6).reshape(1,6)
# 1줄인 1차원 배열
a = np.arange(6).reshape(6,) # 튜플에서 요소가 1개일 때 뒤에 ,를 써줘야한다.

a

array([0, 1, 2, 3, 4, 5])

#### 그룹 연산
* 복잡한 데이터를 어떤 기준에 따라 여러 그룹으로 나누어서 관찰할 수 있으며 이런 방식으로 분할 처리하는 것을 그룹 연산이라 함
* 그룹 연산은 데이터를 집계, 변환, 필터링하는데 효율적이며 판다스 groupby() 메소드를 사용함
* 그룹 객체 만들기(분할)
* 그룹 연산 메소드(적용-결합)

In [ ]:
import pandas as pd
import seaborn as sns

# titanic 데이터셋에서 age, sex 등 5개 열을 선택하여 데이터프레임 만들기
titanic = sns.load_dataset('titanic')
df = titanic[['age', 'sex', 'class', 'fare', 'survived']]
#df = titanic.loc[:, ['age', 'sex', 'class', 'fare', 'survived']]

print("승객 수: ", len(df))
print(df.head())

승객 수:  891
    age     sex  class     fare  survived
0  22.0    male  Third   7.2500         0
1  38.0  female  First  71.2833         1
2  26.0  female  Third   7.9250         1
3  35.0  female  First  53.1000         1
4  35.0    male  Third   8.0500         0


In [ ]:
# 키 피처를 뽑는데 중요한 정보를 얻을 수 있다.
print(titanic.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB
None


과대 적합
- 모델링의 목적은 일반화
- 너무 한 분야에 특화된 모델은 다른 분야에 적합하지 않게된다
- 표본, 기준을 어떻게 잡느냐가 중요
- 모델의 제한조건이 있다면, 그 전제 하에 사용해야함을 명시

In [ ]:
# class 열을 기준으로 분할(1개 열을 기준)

df1 = df[['age', 'class', 'fare', 'survived']]
gr = df1.groupby(['class'])
list(gr)

[(('First',),
        age  class     fare  survived
  1    38.0  First  71.2833         1
  3    35.0  First  53.1000         1
  6    54.0  First  51.8625         0
  11   58.0  First  26.5500         1
  23   28.0  First  35.5000         1
  ..    ...    ...      ...       ...
  871  47.0  First  52.5542         1
  872  33.0  First   5.0000         0
  879  56.0  First  83.1583         1
  887  19.0  First  30.0000         1
  889  26.0  First  30.0000         1
  
  [216 rows x 4 columns]),
 (('Second',),
        age   class     fare  survived
  9    14.0  Second  30.0708         1
  15   55.0  Second  16.0000         1
  17    NaN  Second  13.0000         1
  20   35.0  Second  26.0000         0
  21   34.0  Second  13.0000         1
  ..    ...     ...      ...       ...
  866  27.0  Second  13.8583         1
  874  28.0  Second  24.0000         1
  880  25.0  Second  26.0000         1
  883  28.0  Second  10.5000         0
  886  27.0  Second  13.0000         0
  
  [184 rows x 

In [ ]:
# 그룹객체를 iteration으로 출력: head()메소드로 첫 5행만을 출력
for key, group in gr:
  print(type(key))
  print(type(group))
  #print('key : ', key)
  #print('number : ', len(group))
  #print(group.head())

<class 'tuple'>
<class 'pandas.core.frame.DataFrame'>
<class 'tuple'>
<class 'pandas.core.frame.DataFrame'>
<class 'tuple'>
<class 'pandas.core.frame.DataFrame'>


In [ ]:
avg = gr.mean()
avg.index.name = None
avg

# survived는 0과 1로 이루어져있으니 0.62라면 62%의 의미를 갖는다.

,age,fare,survived
First,38.233441,84.154687,0.629630
Second,29.877630,20.662183,0.472826
Third,25.140620,13.675550,0.242363


In [ ]:
stat = gr.max()
stat

,age,fare,survived
First,80.0,512.3292,1
Second,70.0,73.5000,1
Third,74.0,69.5500,1


In [ ]:
# 개별 그룹 선택하기
gr3 = gr.get_group('Third')
gr3.head()
gr3.describe()

,age,fare,survived
count,355.000000,491.000000,491.000000
mean,25.140620,13.675550,0.242363
std,12.495398,11.778142,0.428949
min,0.420000,0.000000,0.000000
25%,18.000000,7.750000,0.000000
50%,24.000000,8.050000,0.000000
75%,32.000000,15.500000,0.000000
max,74.000000,69.550000,1.000000


In [ ]:
# class 열, sex 열을 기준으로 분할
df2 = df[['age', 'fare', 'survived', 'class', 'sex']]
gr2 = df2.groupby(['class', 'sex'])

In [ ]:
for key, group in gr2:
  print('key: ', key)
  print('number : ', len(group))
  print(group.head())

key:  ('First', 'female')
number :  94
     age      fare  survived  class     sex
1   38.0   71.2833         1  First  female
3   35.0   53.1000         1  First  female
11  58.0   26.5500         1  First  female
31   NaN  146.5208         1  First  female
52  49.0   76.7292         1  First  female
key:  ('First', 'male')
number :  122
     age      fare  survived  class   sex
6   54.0   51.8625         0  First  male
23  28.0   35.5000         1  First  male
27  19.0  263.0000         0  First  male
30  40.0   27.7208         0  First  male
34  28.0   82.1708         0  First  male
key:  ('Second', 'female')
number :  76
     age     fare  survived   class     sex
9   14.0  30.0708         1  Second  female
15  55.0  16.0000         1  Second  female
41  27.0  21.0000         0  Second  female
43   3.0  41.5792         1  Second  female
53  29.0  26.0000         1  Second  female
key:  ('Second', 'male')
number :  108
     age  fare  survived   class   sex
17   NaN  13.0         1 

In [ ]:
# gr2 그룹 객체에 연산 메소드 적용
avg = gr2.mean()
avg

age        fare  survived
class  sex                                    
First  female  34.611765  106.125798  0.968085
       male    41.281386   67.226127  0.368852
Second female  28.722973   21.970121  0.921053
       male    30.740707   19.741782  0.157407
Third  female  21.750000   16.118810  0.500000
       male    26.507589   12.661633  0.135447

In [ ]:
# gr2 그룹 객체에서 개별 그룹 선택하기
# get_group()에 조건을 여러개 넣으려면 튜플 형태로 전달해야한다.
gr3 = gr2.get_group(('Third', 'female'))
gr3.head()

,age,fare,survived,class,sex
2,26.0,7.9250,1,Third,female
8,27.0,11.1333,1,Third,female
10,4.0,16.7000,1,Third,female
14,14.0,7.8542,0,Third,female
18,31.0,18.0000,0,Third,female


데이터 집계(agg),
- 집계 연산을 처리하는 사용자 정의 함수를 그룹 객체에 적용하려면 agg() 메서드 사용
- 모든열에 여러 함수를 매핑 : group객체.agg([함수1,함수2,함수3,…])
- 각 열마다 다른 함수를 매핑 : group객체.agg({‘열1’: 함수1, ‘열2’:함수2, …})

변환(transform)
- 원소의 본래 행 인덱스와 열 이름을 기준으로 연산 결과를 반환
- 데이터 변환 연산 : group객체.transform(매핑함수)

필터링(filter)
- 그룹 객체에 filter() 메서드를 적용할 때 조건식을 가진 함수를 전달하면 조건이 참인 그룹만을 남김
- 그룹 객체 필터링 : group 객체.filter(조건 함수식)

객체에 함수 매핑(apply)
 - 판다스 객체의 개별 원소를 특정 함수에 일대일로 매핑. 사용자가 원하는 대부분의 연산을 그룹 객체에 적용
 - group 객체.apply(매핑함수)

In [ ]:
import pandas as pd
import seaborn as sns

# titanic 데이터셋에서 age, sex 등 5개 열을 선택하여 데이터프레임 만들기
titanic = sns.load_dataset('titanic')
df = titanic.loc[:, ['age','class', 'fare', 'survived']]

# class 열을 기준으로 분할
grouped = df.groupby(['class'])

In [ ]:
mean_all = grouped.mean()
mean_all.index.name = None
print(mean_all)

              age       fare  survived
First   38.233441  84.154687  0.629630
Second  29.877630  20.662183  0.472826
Third   25.140620  13.675550  0.242363


In [ ]:
# 각 그룹에 대한 모든 열의 표준편차를 집계하여 데이터프레임으로 반환

std = grouped.std()
print(std)
print(type(std))

              age       fare  survived
First   14.802856  78.380373  0.484026
Second  14.001077  13.417399  0.500623
Third   12.495398  11.778142  0.428949
<class 'pandas.core.frame.DataFrame'>


In [ ]:
# 각 그룹에 대한 fare 열의 표준편차를 집계하여 시리즈로 반환

std_f = grouped.fare.std()
print(type(std_f))
print(std_f)
print(grouped.fare.mean())

<class 'pandas.core.series.Series'>
First     78.380373
Second    13.417399
Third     11.778142
Name: fare, dtype: float64
First     84.154687
Second    20.662183
Third     13.675550
Name: fare, dtype: float64


In [ ]:
# 그룹 객체에 agg() 메소드 적용 - 사용자 정의 함수를 인수로 전달
def min_max(x): # 최대값 - 최소값
  return x.max() - x.min()

# 각 그룹의 최대값과 최소값의 차이를 계산하여 그룹별로 집계
res = grouped.agg(min_max)
res

,age,fare,survived
First,79.08,512.3292,1
Second,69.33,73.5000,1
Third,73.58,69.5500,1


In [ ]:
# 모든 열에 여러 함수를 매핑 : group객체.agg([함수1, 함수2, 함수3...])
# 각 컬럼마다 다른 함수를 매핑 : group객체.agg({'열1':함수1, '열2':함수2, '열3':함수3...})

res = grouped.agg(['min', 'max', 'mean'])
res = grouped.agg({'fare':['min', 'max'], 'age':'mean'})
res

fare                  age
        min       max       mean
First   0.0  512.3292  38.233441
Second  0.0   73.5000  29.877630
Third   0.0   69.5500  25.140620

In [ ]:
import pandas as pd
import seaborn as sns

# titanic 데이터셋에서 age, sex 등 5개 열을 선택하여 데이터프레임 만들기
titanic = sns.load_dataset('titanic')
df = titanic.loc[:, ['age','class', 'fare', 'survived']]

# class 열을 기준으로 분할
grouped = df.groupby(['class'])

In [ ]:
# 데이터 개수가 200개 이상인 그룹만을 필터링하여 데이터프레임으로 반환
grf = grouped.filter(lambda x:len(x)>=200) # x : 각 group
print(grf.head())

grf.value_counts('class')

    age  class     fare  survived
0  22.0  Third   7.2500         0
1  38.0  First  71.2833         1
2  26.0  Third   7.9250         1
3  35.0  First  53.1000         1
4  35.0  Third   8.0500         0


class
Third     491
First     216
Second      0
Name: count, dtype: int64

#### 함수 매핑
커스텀 함수(custom function)를 DataFrame에 적용하려면 map함수, apply함수, applymap함수를 사용
- map 함수는 Series의 각 요소에 함수를 적용.
- apply 함수는 Series 또는 DataFrame의 행이나 열에 함수를 적용.
- applymap 함수는 DataFrame의 각 요소에 함수를 적용.

In [ ]:
import pandas as pd

s = pd.Series([1,2,3,4,5])

s_m = s.map(lambda x : x + 2)
print(s_m)

0    3
1    4
2    5
3    6
4    7
dtype: int64


In [ ]:
# DataFrame에 적용

df = pd.DataFrame({
    'A': [1, 2, 3],
    'B': [4, 5, 6],
    'C': [7, 8, 9]
})

# 각 열에 대해 최대값을 구하는 함수
df_c = df.apply(lambda x: x.max(), axis=0)

# 각 행에 대해 합계를 구하는 함수
df_i = df.apply(lambda x: x.sum(), axis=1)

apply vs map
- apply는 DataFrame의 행이나 열 단위로 함수를 적용할 수 있지만, map은 Series의 각 요소에만 함수를 적용
- Series의 각 요소를 변환하는 간단한 작업에는 map이 적합하고, 더 복잡한 변환이 필요한 경우나 DataFrame 전체를 다룰 때는 apply가 유용

In [ ]:
# applymap 함수는 DataFrame에만 적용되며, 각 요소에 함수를 적용

df = pd.DataFrame({
    'A': [1, 2, 3],
    'B': [4, 5, 6],
    'C': [7, 8, 9]
})

df_am = df.applymap(lambda x:x+2)
print(df_am)

   A  B   C
0  3  6   9
1  4  7  10
2  5  8  11


In [ ]:
# 함수 적용과 매핑
import numpy as np
import pandas as pd

np.random.seed(10)

frame = pd.DataFrame(np.random.randn(4,3),columns=list('bde'),
                    index = ['Utah','Ohio','Texas','Oregon'])
frame

,b,d,e
Utah,1.331587,0.715279,-1.545400
Ohio,-0.008384,0.621336,-0.720086
Texas,0.265512,0.108549,0.004291
Oregon,-0.174600,0.433026,1.203037


In [ ]:
format = lambda x: '%.2f' %x

In [ ]:
# map함수는 series에만 적용
frame['e'].map(format)

Utah      -1.55
Ohio      -0.72
Texas      0.00
Oregon     1.20
Name: e, dtype: object

In [ ]:
# series의 최대값과 최소값의 차이를 계산
f = lambda x: x.max() - x.min()
frame.apply(f)

b    1.506187
d    0.606730
e    2.748438
dtype: float64

In [ ]:
# 각 행에 대해서 한 번씩 수행
frame.apply(f, axis='columns')

Utah      2.876987
Ohio      1.341422
Texas     0.261220
Oregon    1.377638
dtype: float64

In [ ]:
f1 = lambda x: round(x*10)
frame.apply(f1, axis=0)

,b,d,e
Utah,13.0,7.0,-15.0
Ohio,-0.0,6.0,-7.0
Texas,3.0,1.0,0.0
Oregon,-2.0,4.0,12.0


In [ ]:
frame

,b,d,e
Utah,1.331587,0.715279,-1.545400
Ohio,-0.008384,0.621336,-0.720086
Texas,0.265512,0.108549,0.004291
Oregon,-0.174600,0.433026,1.203037


In [ ]:
f1 = lambda x: round(x*20)
frame.apply(f1, axis = 1)

,b,d,e
Utah,27.0,14.0,-31.0
Ohio,-0.0,12.0,-14.0
Texas,5.0,2.0,0.0
Oregon,-3.0,9.0,24.0


In [ ]:
f1 = lambda x: sum(x)
frame.apply(f1, axis=1)

Utah      0.501465
Ohio     -0.107133
Texas     0.378352
Oregon    1.461463
dtype: float64

- df.apply는 DataFrame의 각 행(row) 또는 열(column)에 함수를 적용합니다. 기본적으로 axis=0으로 설정되어 있어 열 단위로 함수를 적용
- 각 행 또는 열 자체가 Series로 전달되며, 이 Series 전체에 대해 함수를 적용하려고 한다.
- applymap은 DataFrame의 모든 요소에 대해 함수를 적용하므로, 전체 DataFrame에 대한 변환이 필요할 때 유용

In [ ]:
# 여러값을 가진 Series 반환
print(frame,'\n')
def f(x):
  return pd.Series([x.min(), x.max()], index=['min','max'])
frame.apply(f)

               b         d         e
Utah    1.331587  0.715279 -1.545400
Ohio   -0.008384  0.621336 -0.720086
Texas   0.265512  0.108549  0.004291
Oregon -0.174600  0.433026  1.203037 



,b,d,e
min,-0.174600,0.108549,-1.545400
max,1.331587,0.715279,1.203037


In [ ]:
# 실수값을 문자열 포맷으로 변환
format = lambda x: '%.2f' %x
frame.applymap(format)

,b,d,e
Utah,1.33,0.72,-1.55
Ohio,-0.01,0.62,-0.72
Texas,0.27,0.11,0.00
Oregon,-0.17,0.43,1.20


In [ ]:
np.random.seed(0)
data = np.random.randint(1,100,size=(5,5))
df = pd.DataFrame(data, columns=list('abced'))
df

,a,b,c,e,d
0,45,48,65,68,68
1,10,84,22,37,88
2,71,89,89,13,59
3,66,40,88,47,89
4,82,38,26,78,73


In [ ]:
#df['f'] = df.apply(lambda x: x.a+x.b+x.c+x.d+x.e, axis=1)
#df['f'] = df.apply(lambda x: x.sum(), axis=1)
#print(df)
df['g'] = df.sum(axis=1)
print(df)

    a   b   c   e   d    g
0  45  48  65  68  68  294
1  10  84  22  37  88  241
2  71  89  89  13  59  321
3  66  40  88  47  89  330
4  82  38  26  78  73  297


In [ ]:
# 표준화 : 데이터의 평균을 0, 표준편차를 1로 변환하는 과정.
# 이 과정을 통해 데이터의 스케일을 조정하고 비교 가능한 형태로 변환
def z_score(x):
  return (x - x.mean()) / x.std()

# 베트남 동과 US달라로 비교해보자면
# 값에서 평균값을 빼고 표준편차로 나누어주는게 공식
# 표준화하는 함수가 따로 존재

In [ ]:
# 원소의 보내 행 인덱스와 열 이름을 기준으로 연산 결과를 반환
df_zscore = df.a.transform(z_score)
df_zscore

0   -0.344827
1   -1.576351
2    0.570020
3    0.394088
4    0.957070
Name: a, dtype: float64

In [ ]:
df_zscore = df.apply(lambda x:z_score(x))
df_zscore

,a,b,c,e,d,g
0,-0.344827,-0.477299,0.215257,0.754401,-0.570413,-0.074984
1,-1.576351,0.978867,-1.107037,-0.451085,0.971244,-1.603498
2,0.570020,1.181112,0.953282,-1.384365,-1.264159,0.703693
3,0.394088,-0.800891,0.922531,-0.062219,1.048327,0.963252
4,0.957070,-0.881789,-0.984033,1.143268,-0.184999,0.011536


## .agg()에 적용할 수 있는 통계함수 문자열 표

- count	데이터의 개수
- sum	합계
- mean	평균
- median	중앙값
- var, std	분산, 표준편차
- min, max	최소, 최대값
- unique, nunique	고유값, 고유값 개수
- prod	곲
- first, last	첫째, 마지막값

####Pivot
pivot 함수의 구성 형식
- index: 새로운 DataFrame에서 인덱스로 사용할 기존 열 또는 열들의 이름.
- columns: 새로운 DataFrame에서 열로 사용할 기존 열 또는 열들의 이름.
- values: 피벗할 때 사용할 값이 있는 열의 이름.

기능
- 긴 형식의 데이터를 넓은 형식으로 변환하여 특정 차원에서 데이터를 재구성.
- 피벗된 데이터를 통해 시간 경과에 따른 변화나 여러 카테고리의 비교를 쉽게 할 수 있다.
- 넓은 형식의 데이터는 시각화 도구에서 더 쉽게 다룰 수 있다.

In [5]:
# 날짜(date)를 인덱스로, 도시(city)를 열로, 온도(temperature)를 값으로 사용하여 피벗 테이블을 ㅐㅅㅇ성
import pandas as pd

data  = {
  'date':['2021-01-01','2021-01-01','2021-01-02','2021-01-02'],
  'city':['New York','Los Angeles','New York','Los Angeles'],
  'temperature':[32, 75, 30, 78]
}

df = pd.DataFrame(data)

pivot_df = df.pivot(index='date', columns='city', values='temperature')
pivot_df.index.name = None
pivot_df.columns.name = None
pivot_df

,Los Angeles,New York
2021-01-01,75,32
2021-01-02,78,30


In [ ]:
# 중복 값을 평균으로 처리하여 피벗 테이블 생성
data  = {
  'date':['2021-01-01','2021-01-01','2021-01-01','2021-01-02','2021-01-02'],
  'city':['New York','Los Angeles','New York','New York','Los Angeles'],
  'temperature':[32, 75, 30, 30, 78]
}

df = pd.DataFrame(data)

# 피벗 테이블 생성 : 중복 값의 평균 계산
pivot_table = df.pivot_table(index='date', columns='city', values='temperature', aggfunc='mean')
pivot_table

In [11]:
data  = {
  'date':['2021-01-01','2021-01-01','2021-01-02','2021-01-02'],
  'city':['New York','Los Angeles','New York','Los Angeles'],
  'temperature':[32, 75, 30, 30],
  'humidity':[80,20,85,15]
}

df = pd.DataFrame(data)

# 다중 인덱스 및 다중 값 피벗 테이블 생성
pivot_multi = df.pivot(index='date', columns='city')

pivot_multi.index.name=None
pivot_multi

temperature             humidity         
city       Los Angeles New York Los Angeles New York
2021-01-01          75       32          20       80
2021-01-02          30       30          15       85

In [16]:
data = {
    'Year': [2020, 2020, 2021, 2021, 2022, 2022, 2022, 2023, 2023],
    'Product': ['A', 'B', 'A', 'B', 'A', 'B', 'C', 'A', 'C'],
    'Sales': [100, 150, 200, 250, 300, 350, 400, 500, 450]
}

df = pd.DataFrame(data)

# 피벗 테이블 생성 : 각 여도오 ㅏ각 제품별로 총 매출액 계산
p = df.pivot_table(index='Year', columns='Product', values='Sales', aggfunc='sum', fill_value=0)
p.columns.name=None
p.index.name=None
p

,A,B,C
2020,100,150,0
2021,200,250,0
2022,300,350,400
2023,500,0,450


In [22]:
# Q. 주어진 직원 근무 시간 데이터에서 각 직원별로 각 월의 총 근무 시간을 계산하세요.

data = {
    'Employee': ['John', 'John', 'John', 'Anna', 'Anna', 'Anna', 'Peter', 'Peter', 'Peter'],
    'Month': ['Jan', 'Feb', 'Mar', 'Jan', 'Feb', 'Mar', 'Jan', 'Feb', 'Mar'],
    'Hours': [160, 170, 175, 150, 165, 180, 155, 160, 170]
}

df = pd.DataFrame(data)
print(df)

p = df.pivot_table(index='Employee', columns='Month', values='Hours', aggfunc='sum', fill_value=0)
p.columns.name=None
p.index.name=None
p

  Employee Month  Hours
0     John   Jan    160
1     John   Feb    170
2     John   Mar    175
3     Anna   Jan    150
4     Anna   Feb    165
5     Anna   Mar    180
6    Peter   Jan    155
7    Peter   Feb    160
8    Peter   Mar    170


,Feb,Jan,Mar
Anna,165,150,180
John,170,160,175
Peter,160,155,170
